In [ ]:
import os, time, datetime, operator, sys
import tensorflow as tf
import random
import numpy as np
from sklearn import metrics
from operator import itemgetter
sys.path.append('../')
v_trainfile = '../data/output_all.txt'
trainfile_1='../data/output_1.txt'
trainfile_2='../data/output_2.txt'
trainfile_3='../data/output_3.txt'
#vectorsfile = '../data/vectors.txt'
##########################################################################
#  embedding_lookup + cnn + cosine margine ,  batch
##########################################################################
class QACNN(object):
    def __init__(self, _margin, sequence_length, batch_size,vocab_size, embedding_size,filter_sizes, num_filters, l2_reg_lambda=0.0):
        self.L, self.B, self.V, self.E, self.FS, self.NF = sequence_length, batch_size,vocab_size, embedding_size, filter_sizes, num_filters 

        #用户问题,字向量使用embedding_lookup
        self.q = tf.placeholder(tf.int32, [self.B, self.L], name="q")
        #待匹配正向问题
        self.qp = tf.placeholder(tf.int32, [self.B, self.L], name="qp")
        #负向问题
        self.qn = tf.placeholder(tf.int32, [self.B, self.L], name="qn")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            W = tf.get_variable(initializer=tf.random_uniform([self.V, self.E], -1.0, 1.0),name='We')
            self.qe = tf.nn.embedding_lookup(W, self.q)
            self.qpe = tf.nn.embedding_lookup(W, self.qp)
            self.qne = tf.nn.embedding_lookup(W, self.qn)
        self.qe = tf.expand_dims(self.qe, -1)
        self.qpe = tf.expand_dims(self.qpe, -1)
        self.qne = tf.expand_dims(self.qne, -1)
        with tf.variable_scope('shared-conv') as scope:
            self.qe = self.conv(self.qe)
            scope.reuse_variables()
            #tf.get_variable_scope().reuse_variables()
            self.qpe = self.conv(self.qpe)
            scope.reuse_variables()
            #tf.get_variable_scope().reuse_variables()
            self.qne = self.conv(self.qne)
        self.cos_q_qp = self.cosine(self.qe, self.qpe)
        self.cos_q_qn = self.cosine(self.qe, self.qne)
        zero = tf.constant(0, shape=[self.B], dtype=tf.float32)
        margin = tf.constant(_margin, shape=[self.B], dtype=tf.float32)
        with tf.name_scope("loss"):
            self.losses = tf.maximum(zero, tf.subtract(margin, tf.subtract(self.cos_q_qp, self.cos_q_qn)))
            self.loss = tf.reduce_sum(self.losses) + l2_reg_lambda * l2_loss
            print('loss ', self.loss)

        # Accuracy
        with tf.name_scope("accuracy"):
            self.correct = tf.equal(zero, self.losses)
            self.accuracy = tf.reduce_mean(tf.cast(self.correct, "float"), name="accuracy")

        for v in tf.trainable_variables():
            print(v)

    def conv(self, tensor):
        pooled = []   
        with tf.variable_scope("my-conv-shared"):
            for i, fs in enumerate(self.FS):
                filter_shape = [fs, self.E, 1, self.NF]
                W = tf.get_variable(initializer=tf.truncated_normal(filter_shape, stddev=0.1), name="W-%s" % str(fs))
                b = tf.get_variable(initializer=tf.constant(0.1, shape=[self.NF]), name="b-%s" % str(fs))
                conv = tf.nn.conv2d(tensor, W, strides=[1, 1, 1, 1], padding='VALID',name="conv")
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                output = tf.nn.max_pool(h, ksize=[1, self.L - fs + 1, 1, 1], strides=[1, 1, 1, 1], padding='VALID',name="pool")
                pooled.append(output)
            num_filters_total = self.NF * len(self.FS)
            pooled = tf.reshape(tf.concat(pooled, 3), [-1, num_filters_total])
            pooled = tf.nn.dropout(pooled, self.dropout_keep_prob)
            return pooled

    def cosine(self, v1, v2):
        l1 = tf.sqrt(tf.reduce_sum(tf.multiply(v1, v1), 1))
        l2 = tf.sqrt(tf.reduce_sum(tf.multiply(v2, v2), 1))
        a = tf.reduce_sum(tf.multiply(v1, v2), 1)
        cos = tf.div(a, tf.multiply(l1, l2), name='score')
        return tf.clip_by_value(cos, 1e-5, 0.99999)

#build a vocabulary list based on splited txt
def build_vocab():
    global v_trainfile
    code, vocab = int(0), {}
    vocab['UNKNOWN'] = code
    code += 1
    with open(v_trainfile,encoding='utf-8') as f:
        lines=f.readlines()
    for line in lines:
        items = line.encode('utf-8').decode('utf-8-sig').strip().split()
        for i in range(0, len(items)):
            word=(str)(items[i])
            if not word in vocab and word!='':
                vocab[word] = code
                code += 1
    return vocab
'''
#if not using tensorflow to train w2v model
#load trained w2v vectors
def load_vectors():
    global vectorsfile
    vectors = {}
    with open(vectorsfile,encoding='utf-8') as f:
        contents=f.read().encode('utf-8').decode('utf-8-sig').split()
    for i in range(0,int(len(contents)/101)):
        vec = []
        
        for j in range(1,101):
            vec.append(float(contents[j+i*101]))
        vectors[contents[i*101]]=vec
    return vectors
#match words' vector with vocab (brute force) 
def load_word_embeddings(vocab, dim):
    vectors = load_vectors()
    embeddings = [] #brute initialization
    for i in range(0, len(vocab)):
        vec = []
        for j in range(0, dim):
            vec.append(0.01)
        embeddings.append(vec)
    for word, code in vocab.items():
        if word in vectors:
            embeddings[code] = vectors[word]
    return np.array(embeddings, dtype='float32')
'''
#read txts pay attention to encoding
def read_txt(path):
    x=[]
    with open(path,'r',encoding='utf-8') as f:
        lines=f.readlines()
    for line in lines:
        x.append(line.encode('utf-8').decode('utf-8-sig').strip())
    return x
#load trainList as [q, s_pq, s_nq, id]
def load_train_list():
    global trainfile_1
    global trainfile_2
    global trainfile_3
    c1=read_txt(trainfile_1)
    c2=read_txt(trainfile_2)
    c3=read_txt(trainfile_3)
    trainList = []
    for i in range(0,len(c1)):
        trainList.append([c1[i]]+[c2[i]]+[c3[i]]+[i])#[q,s_pq,s_nq,id]
    return trainList
#load testList as [flag, test_q, ans_q, id]
def load_test_list(trainList):
    testList = []
    test_id=[]
    #randomly match pos and neg question pair
    for i in range(0,50):
        test_id.append(random.randint(0, len(trainList)-1))
        testList.append([1,trainList[test_id[i]][0],trainList[test_id[i]][1],trainList[test_id[i]][3]])
        testList.append([0,trainList[test_id[i]][0],trainList[test_id[i]][2],trainList[test_id[i]][3]])
        for j in range(0,18):
            testList.append([0,trainList[test_id[i]][0],trainList[random.randint(0, len(trainList)-1)][2],trainList[test_id[i]][3]])
        del trainList[test_id[i]]
    return testList

# encode string(splited words) as code value in vocab
def encode_sent(vocab, string, size):
    x = []
    words = string.split()
    if(len(words)<size):
        words+=['0']*(size-len(words))
    for i in range(0, size):
        if words[i] in vocab:
            x.append(vocab[words[i]])
        else:
            x.append(vocab['UNKNOWN'])
    return x
#turn elements in trainList into coded value for train 
def load_data(trainList, vocab, batch_size):
    train_1, train_2, train_3 = [], [], []
    for i in range(0, batch_size):
        seq=trainList[random.randint(0, len(trainList)-1)]
        train_1.append(encode_sent(vocab, seq[0], 100))
        train_2.append(encode_sent(vocab, seq[1], 100))
        train_3.append(encode_sent(vocab, seq[2], 100))
    return np.array(train_1, dtype='float32'), np.array(train_2, dtype='float32'), np.array(train_3, dtype='float32')
#generate a batch of test data
def gen_test_batch_qpn(testList,start,end,vocab):
    f,tq,aq,id=[],[],[],[]
    if end<len(testList):
        for i in range(start,end):
            s=testList[i]
            f.append(s[0])
            tq.append(encode_sent(vocab,s[1],100))
            aq.append(encode_sent(vocab,s[2],100))
            id.append(s[3])
    else:
        for i in range(start,len(testList)):
            s=testList[i]
            f.append(s[0])
            tq.append(encode_sent(vocab,s[1],100))
            aq.append(encode_sent(vocab,s[2],100))
            id.append(s[3])
        for i in range(0,end-len(testList)):
            s=testList[i]
            f.append(s[0])
            tq.append(encode_sent(vocab,s[1],100))
            aq.append(encode_sent(vocab,s[2],100))
            id.append(s[3])
    tq=np.array(tq, dtype='float32')
    aq=np.array(aq, dtype='float32')
    return f,id,tq,aq
#evaluate auc
def eval_auc(y,yp):
    auc = metrics.roc_auc_score(y, yp)
    print('auc: ' + str(auc))
    return auc
#evaluate  top1_prec and F1 in a batch test
def eval_top1_prec(y, g, yp):
    _list = [(_y, _g, _yp) for _y, _g, _yp in zip(y, g, yp)]
    _dict = {}
    for _y, _g, _yp in _list:
        if not _g in _dict: _dict[_g] = []
        _dict[_g].append((_y, _g, _yp))
    positive, gc = 0 , 0
    P=1
    for _, group in _dict.items():
        group = sorted(group, key=itemgetter(2), reverse=True)
        gc += 1
        if group[0][0] == 1: 
            positive += 1
    prec = positive / gc
    R=prec
    F1=2*R/(1+R)
    print('top1 precision ' + str(positive) + '/' + str(gc) + ': '+ str(positive / gc))
    print('F1: %f'%F1)
    return prec,F1
    
# Parameters
# ==================================================

# Model Hyperparameters,restart after loading once
tf.app.flags.DEFINE_string('f', '', 'kernel')
tf.flags.DEFINE_float("margin", 0.05, "CNN model margin")
tf.flags.DEFINE_integer("sequence_length", 100, "Max sequence lehgth(default: 100)")
tf.flags.DEFINE_integer("embedding_dim", 100, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "1,2,3,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 256, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 1.0, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0, "L2 regularizaion lambda (default: 0.0)")
# Training parameters
tf.flags.DEFINE_integer("batch_size", 256, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 20000, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 500, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 500, "Save model after this many steps (default: 100)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
#FLAGS = tf.flags.FLAGS #new veision
FLAGS._parse_flags() #old version
FLAGS.flag_values_dict()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")

# Data Preparatopn
# ==================================================

# Load data
print("Loading data...")
vocab = build_vocab()
trainList=load_train_list()
print("Load done...")

# Training
# ==================================================

#define session_conf
with tf.Graph().as_default():
    with tf.device("/gpu:1"):
        session_conf = tf.ConfigProto(allow_soft_placement=FLAGS.allow_soft_placement,log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = QACNN(
            _margin=FLAGS.margin,
            sequence_length=FLAGS.sequence_length,
            batch_size=FLAGS.batch_size,
            vocab_size=len(vocab),
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-1)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        #define model saver
        saver = tf.train.Saver(tf.global_variables(),max_to_keep=100)

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(q, qp, qn):
            feed_dict = {
              cnn.q: q, cnn.qp: qp, cnn.qn: qn,
              #cnn.input_x_1: q, cnn.input_x_2: qp, cnn.input_x_3: qn,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _,step, summaries, loss, accuracy, cos1, cos2 = sess.run(
            [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy, cnn.cos_q_qp, cnn.cos_q_qn],feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def test_step(testList):
            yp, y, group, of = [], [], [], open('../data/predict.txt', 'w')
            start=0
            end=start+FLAGS.batch_size
            for i in range(0, len(testList), FLAGS.batch_size):
                f, g, q1, q2 = gen_test_batch_qpn(testList,i,i+FLAGS.batch_size,vocab)
                feed_dict = {
                    cnn.q: q1, cnn.qp: q2, cnn.qn: q2,
                    #cnn.input_x_1: q1, cnn.input_x_2: q2, cnn.input_x_3: q2,
                    cnn.dropout_keep_prob: 1.0}
                cos = sess.run([cnn.cos_q_qp], feed_dict)
                yp.extend(cos[0])
                y.extend(f)
                group.extend(g)
            y, g, yp = y[:len(testList)], group[:len(testList)], yp[:len(testList)]
            auc = eval_auc(y[:len(testList)],yp[:len(testList)])
            top1_prec,F1 = eval_top1_prec(y, g, yp)
            for p in yp[:len(testList)]: 
                of.write(str(p) + '\n')
            of.write(str(top1_prec) + '\n')
            of.close()
            return auc,top1_prec,F1

        # Generate batches
        # Training loop. For each batch...
        #define average accuracy ,lowest and highest for accuracy and F1(recall)
        aver=0
        la=1
        ha=0
        lf=1
        hf=0
        #log id for best model
        idx=0
        for i in range(FLAGS.num_epochs):
            try:
                trainList=load_train_list()
                #load shuffle testList 
                testList=load_test_list(trainList)
                q, qp, qn = load_data(trainList,vocab, FLAGS.batch_size)
                train_step(q, qp, qn)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % FLAGS.evaluate_every == 0:
                    auc,top1_prec,F1= test_step(testList)
                    if la>top1_prec:
                        la=top1_prec
                    if lf>F1:
                        lf=F1
                    if ha<top1_prec:
                        ha=top1_prec
                        idx=current_step/FLAGS.checkpoint_every
                    if hf<F1:
                        hf=F1
                    aver+=top1_prec
                if current_step % FLAGS.checkpoint_every == 0:
                    path = saver.save(sess, './model/model.ckpt', global_step=int(current_step/FLAGS.checkpoint_every))
                    print("Saved model checkpoint to {}\n".format(path))
            except Exception as e:
                print(e)
        print("average top1_prec= {}".format((aver*FLAGS.evaluate_every)/FLAGS.num_epochs))
        print("lowest top1_prec={}".format(la))
        print("highest top1_prec={}".format(ha))
        print("id :{}".format(idx))
        print("lowest F1={}".format(lf))
        print("highest F1={}".format(hf))

    